# SAS On-the-Job | Part 3
## Role: Data Analyst within the Department of Health and Human Services

### The Final Chapter! Map Time!
R you ready for the R portion of our analysis? And do you love a bad pun? Then you're in just the right place!

In Part 3 of our HHS adventures, we return to our state-level data on labor supply by prime-aged women aged 25-54. Our analytical objective in this section is to use the cool geographic mapping procedures in R to examine changes in the unemployment rate and the labor force participation rate from 2015 to 2022. As is the consistent theme, the general goal here is to see whether pre-COVID trends are back.  Because if they're not, HHS leadership could advocate for policies to support segments of women in the labor force, for example, women with small children.

Let's finish our analysis strong!

#### Prepare the R Environment
The first step is to load the requisite libraries in R. Note that a library is a different thing in R than SAS. In R, it's a package of tools, rather than a pointer to a data location. That's a fun fact for your next trivia night!

In [ ]:
# Load required libraries
library(ggplot2)
library(reshape2)
library(grid)
library(scales)
library(mapproj)
library(viridis)
library(maps)
library(haven)

#### Access Map Data in R
Next, we'll want to leverage data that's already included as part of the R install. We can then link this data to our data set - and ensure that data gets assigned properly in our upcoming maps.

In [ ]:
# R comes with some pre-installed map data. Let's leverage that in this analysis

# Load the U.S. state map data
map_data <- map_data("state")

# Ensure Proper Casing for Merge
map_data$region = toupper(map_data$region)

It's never a bad thing to explore your data.  So, let's check out the first 10 observations in *map_data* to ensure that it's in a form we recognize.

In [ ]:
head(map_data,10)

And that's just a big ole data set, which is why we limited it.

Now let's prepare *covid_labor_supply2* for a merge.  Note that we need to match case on the *State_Name*, which is why we've got that extra line of code.

In [ ]:
# Prepare the data from Part 2 for a merge
data1 <- read_sas('covid_labor_supply2.sas7bdat')

# Adjust Casing for Merge
data1$State_Name = toupper(data1$State_Name)

Let's explore the data one more time - to ensure that we've got the right one:

In [ ]:
data1

#### Merge Time, Merge Time!
Are you ready to pull together the two pieces? Well, I used an exclamation point, so you know that I’m excited. Use the following code to combine *map_data* from R with our aggregated ACS data set.  Data populated all the way across the row tells us that the merge was done properly - so check for that:

In [ ]:
# Merge the data with map data
state_map_data <- merge(map_data, data1, by.x = "region", by.y = "State_Name", all.x = TRUE)

# Let's check out the first 20 observations of the merged data!
head(state_map_data,20)

Data look good to me. How are things on your end? Do you see data all the way across the row?

#### One More Housekeeping Item
The last analysis preparation item is to think a bit more about how the map of the U.S. will appear. And while the exact details of *theme_map* can be handled on another day, let's steal some code to create a pretty theme map for our upcoming plots.

In [ ]:
# Check this out (but don't ask me what each piece means)
# Source Code: https://socviz.co/index.html#preface ==> this book is great! 
theme_map <- function(base_size=20, base_family="") {
  require(grid)
  theme_bw(base_size=base_size, base_family=base_family) %+replace%
    theme(axis.line=element_blank(),
          axis.text=element_blank(),
          axis.ticks=element_blank(),
          axis.title=element_blank(),
          panel.background=element_blank(),
          panel.border=element_blank(),
          panel.grid=element_blank(),
          panel.spacing=unit(0, "lines"),
          plot.background=element_blank(),
          legend.justification = c(0,0),
          legend.position = c(0,0),
          legend.key.width = unit(20,"lines")
    )
}

options(repr.plot.width = 24, repr.plot.height = 16)

Data are now loaded into R - and the theme map is ready to go. Let's get our mapping on!

#### Unemployment Rate Analysis
Let's now explore the unemployment rate across time in the U.S. We'll just focus on the continental U.S. (to simplify) but let's examine the trends for all prime-aged women:

In [ ]:
p0 <- ggplot(data = subset(state_map_data),
             mapping = aes(x = long, y = lat,
                           group = group,
                           fill = UE_Women))

p1 <- p0 + geom_polygon(color = "gray90", linewidth = 0.10) +
  coord_map(projection = "albers", lat0 = 39, lat1 = 45) 

p2 <- p1 + scale_fill_viridis_c(option = "plasma", label=function(x) sprintf("%.2f",x))

p2 + theme_map() + facet_wrap(~ Year, ncol = 3) +
  theme(legend.position = "bottom", strip.background = element_blank()) +
  labs(fill = "Unemployment Rate ",
       title = "Annual Unemployment Rate | 2015 to 2022 | All Women") 

What do you see in these trends? Again, I see that trends have returned to the pre-pandemic levels.  And, dare I say it, it looks like employment prospects is better in 2022 than it was in the earlier period.

Because it's easy, let's just modify the code above and change UE_Women to UE_Women_YoungKids. The code:

In [ ]:
p0 <- ggplot(data = subset(state_map_data),
             mapping = aes(x = long, y = lat,
                           group = group,
                           fill = UE_Women_YoungKids))

p1 <- p0 + geom_polygon(color = "gray90", linewidth = 0.10) +
  coord_map(projection = "albers", lat0 = 39, lat1 = 45) 

p2 <- p1 + scale_fill_viridis_c(option = "plasma", label=function(x) sprintf("%.2f",x))

p2 + theme_map() + facet_wrap(~ Year, ncol = 3) +
  theme(legend.position = "bottom",
        strip.background = element_blank()) +
  labs(fill = "Unemployment Rate ",
       title = "Unemployment Rate | 2015 to 2022 | Women with Young Kids (Age<5)") 

Again, a similar story plays out. Unemployment rates have returned to their pre-pandemic norms... and might be even better than they were in the early period.

#### Labor Force Participation Rate Analysis
Because this code is so easy to modify after it's written, let's examine the labor force participant rate for all women. The story shouldn't change, but it is interesting to see the variation over time within states.

In [ ]:
# Labor Force Participation Rate
r0 <- ggplot(data = subset(state_map_data),
             mapping = aes(x = long, y = lat,
                           group = group,
                           fill = LFP_Women) )
       
r1 <- r0 + geom_polygon(color = "gray90", linewidth = 0.10) +
  coord_map(projection = "albers", lat0 = 39, lat1 = 45) 

r2 <- r1 + scale_fill_viridis_c(option = "plasma", label=function(x) sprintf("%.2f",x))

r2 + theme_map() + facet_wrap(~ Year, ncol = 3) +
  theme(legend.position = "bottom",
        strip.background = element_blank()) +
  labs(fill = "Labor Force Participation Rate ",
       title = "Labor Force Participation Rate | 2015 to 2022 | All Women") 

The story repeats. Things were good. Then they got really bad. And they're either the same as the pre-pandemic levels - or better now.

My recommendation is HHS leadership to take no action on designing supports specifically targeting women affected by the pandemic. Whether segments of women - that is, those with young children - should receive other supports to encourage their labor supply is a great research question for another time.  For now, congrats on completing your first day as a Department of Health and Human Services Policy Analyst!